In [1]:
library(ggplot2);
library(plyr);
library(dplyr);
library(Hmisc);
library(broom);
library(xts);
library(lubridate)
library(cluster)
options(warn=-1)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: lattice
Loading required package: survival
Loading required package: Formula

Attaching package: ‘Hmisc’

The following objects are masked from ‘package:dplyr’:

    combine, src, summarize

The following objects are masked from ‘package:plyr’:

    is.discrete, summarize

The following objects are masked from ‘package:base’:

    format.pval, round.POSIXt, trunc.POSIXt, units

Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Attaching package: ‘xts’

The following objects are masked from ‘package:dplyr’:

    first, last


Attaching packa

In [2]:
data <- read.csv('consumer_data.csv')

In [3]:
str(data)

'data.frame':	1648824 obs. of  20 variables:
 $ fsn_id                         : Factor w/ 21219 levels "ACCCX3S58G7B5F6P",..: 1 1 3 3 3 6 6 6 6 6 ...
 $ order_date                     : Factor w/ 1155192 levels "2015-05-19 13:42:09",..: 261290 272488 276287 200433 266471 258475 241221 218942 219916 291196 ...
 $ Year                           : int  2015 2015 2015 2015 2015 2015 2015 2015 2015 2015 ...
 $ Month                          : int  10 10 10 10 10 10 10 10 10 10 ...
 $ order_id                       : num  3.42e+15 1.42e+15 2.42e+15 4.42e+15 4.42e+15 ...
 $ order_item_id                  : num  3.42e+15 1.42e+15 2.42e+15 4.42e+15 4.42e+15 ...
 $ gmv                            : num  6400 6900 1990 1690 1618 ...
 $ units                          : int  1 1 1 1 1 1 1 1 1 1 ...
 $ deliverybdays                  : Factor w/ 143 levels "-10","-12","-13",..: 24 24 24 24 24 24 24 24 24 24 ...
 $ deliverycdays                  : Factor w/ 171 levels "-11","-115","-14",..: 37 37 37 3


#### Creating a Week Column

In [4]:
data$order_date <- date(data$order_date)

In [5]:
head(data)

fsn_id,order_date,Year,Month,order_id,order_item_id,gmv,units,deliverybdays,deliverycdays,s1_fact.order_payment_type,sla,cust_id,pincode,product_analytic_super_category,product_analytic_category,product_analytic_sub_category,product_analytic_vertical,product_mrp,product_procurement_sla
ACCCX3S58G7B5F6P,2015-10-17,2015,10,3.419301e+15,3.419301e+15,6400,1,\N,\N,COD,5,-1.012991e+18,-7.791756e+18,CE,CameraAccessory,CameraAccessory,CameraTripod,7190,0
ACCCX3S58G7B5F6P,2015-10-19,2015,10,1.420831e+15,1.420831e+15,6900,1,\N,\N,COD,7,-8.990325e+18,7.335411e+18,CE,CameraAccessory,CameraAccessory,CameraTripod,7190,0
ACCCX3S5AHMF55FV,2015-10-20,2015,10,2.421913e+15,2.421913e+15,1990,1,\N,\N,COD,10,-1.040443e+18,-7.477688e+18,CE,CameraAccessory,CameraAccessory,CameraTripod,2099,3
ACCCX3S5AHMF55FV,2015-10-14,2015,10,4.416592e+15,4.416592e+15,1690,1,\N,\N,Prepaid,4,-7.604961e+18,-5.835932e+18,CE,CameraAccessory,CameraAccessory,CameraTripod,2099,3
ACCCX3S5AHMF55FV,2015-10-17,2015,10,4.419525e+15,4.419525e+15,1618,1,\N,\N,Prepaid,6,2.894557e+18,5.347354e+17,CE,CameraAccessory,CameraAccessory,CameraTripod,2099,3
ACCCX3S5JGAJETYR,2015-10-17,2015,10,3.419189e+15,3.419189e+15,3324,1,\N,\N,Prepaid,5,-7.641546e+18,-1.919053e+18,CE,CameraAccessory,CameraAccessory,CameraTripod,4044,5


In [6]:
data$week_year <- week(data$order_date)


In [7]:
head(data)

fsn_id,order_date,Year,Month,order_id,order_item_id,gmv,units,deliverybdays,deliverycdays,⋯,sla,cust_id,pincode,product_analytic_super_category,product_analytic_category,product_analytic_sub_category,product_analytic_vertical,product_mrp,product_procurement_sla,week_year
ACCCX3S58G7B5F6P,2015-10-17,2015,10,3.419301e+15,3.419301e+15,6400,1,\N,\N,⋯,5,-1.012991e+18,-7.791756e+18,CE,CameraAccessory,CameraAccessory,CameraTripod,7190,0,42
ACCCX3S58G7B5F6P,2015-10-19,2015,10,1.420831e+15,1.420831e+15,6900,1,\N,\N,⋯,7,-8.990325e+18,7.335411e+18,CE,CameraAccessory,CameraAccessory,CameraTripod,7190,0,42
ACCCX3S5AHMF55FV,2015-10-20,2015,10,2.421913e+15,2.421913e+15,1990,1,\N,\N,⋯,10,-1.040443e+18,-7.477688e+18,CE,CameraAccessory,CameraAccessory,CameraTripod,2099,3,42
ACCCX3S5AHMF55FV,2015-10-14,2015,10,4.416592e+15,4.416592e+15,1690,1,\N,\N,⋯,4,-7.604961e+18,-5.835932e+18,CE,CameraAccessory,CameraAccessory,CameraTripod,2099,3,41
ACCCX3S5AHMF55FV,2015-10-17,2015,10,4.419525e+15,4.419525e+15,1618,1,\N,\N,⋯,6,2.894557e+18,5.347354e+17,CE,CameraAccessory,CameraAccessory,CameraTripod,2099,3,42
ACCCX3S5JGAJETYR,2015-10-17,2015,10,3.419189e+15,3.419189e+15,3324,1,\N,\N,⋯,5,-7.641546e+18,-1.919053e+18,CE,CameraAccessory,CameraAccessory,CameraTripod,4044,5,42


In [8]:
data$week_month <- (ceiling(as.numeric(format(data$order_date,"%d"))/7))

In [9]:
data$date <- as.Date(data$order_date)

In [10]:
head(data)

fsn_id,order_date,Year,Month,order_id,order_item_id,gmv,units,deliverybdays,deliverycdays,⋯,pincode,product_analytic_super_category,product_analytic_category,product_analytic_sub_category,product_analytic_vertical,product_mrp,product_procurement_sla,week_year,week_month,date
ACCCX3S58G7B5F6P,2015-10-17,2015,10,3.419301e+15,3.419301e+15,6400,1,\N,\N,⋯,-7.791756e+18,CE,CameraAccessory,CameraAccessory,CameraTripod,7190,0,42,3,2015-10-17
ACCCX3S58G7B5F6P,2015-10-19,2015,10,1.420831e+15,1.420831e+15,6900,1,\N,\N,⋯,7.335411e+18,CE,CameraAccessory,CameraAccessory,CameraTripod,7190,0,42,3,2015-10-19
ACCCX3S5AHMF55FV,2015-10-20,2015,10,2.421913e+15,2.421913e+15,1990,1,\N,\N,⋯,-7.477688e+18,CE,CameraAccessory,CameraAccessory,CameraTripod,2099,3,42,3,2015-10-20
ACCCX3S5AHMF55FV,2015-10-14,2015,10,4.416592e+15,4.416592e+15,1690,1,\N,\N,⋯,-5.835932e+18,CE,CameraAccessory,CameraAccessory,CameraTripod,2099,3,41,2,2015-10-14
ACCCX3S5AHMF55FV,2015-10-17,2015,10,4.419525e+15,4.419525e+15,1618,1,\N,\N,⋯,5.347354e+17,CE,CameraAccessory,CameraAccessory,CameraTripod,2099,3,42,3,2015-10-17
ACCCX3S5JGAJETYR,2015-10-17,2015,10,3.419189e+15,3.419189e+15,3324,1,\N,\N,⋯,-1.919053e+18,CE,CameraAccessory,CameraAccessory,CameraTripod,4044,5,42,3,2015-10-17


#### Missing Value Imputation for GMV 

In [11]:
summary(data)

              fsn_id          order_date              Year     
 ACCE6VGHYKYFX4WB:  37622   Min.   :2015-05-19   Min.   :2015  
 SELDXMC3JNMTQMGV:  17859   1st Qu.:2015-10-16   1st Qu.:2015  
 CAMDA6RJERF8HS4G:  16741   Median :2016-01-07   Median :2016  
 ACCDXEGXG8HJUCSK:  14281   Mean   :2016-01-09   Mean   :2016  
 ACCECYHH4QDNZXEX:  13640   3rd Qu.:2016-04-01   3rd Qu.:2016  
 ACCE9C3WYBMPJ98R:  13281   Max.   :2016-07-25   Max.   :2016  
 (Other)         :1535400                                      
     Month           order_id         order_item_id            gmv        
 Min.   : 1.000   Min.   :7.633e+07   Min.   :1.080e+08   Min.   :     0  
 1st Qu.: 3.000   1st Qu.:1.603e+15   1st Qu.:1.607e+15   1st Qu.:   340  
 Median : 6.000   Median :2.612e+15   Median :2.619e+15   Median :   750  
 Mean   : 6.586   Mean   :2.842e+15   Mean   :1.376e+16   Mean   :  2461  
 3rd Qu.:10.000   3rd Qu.:3.611e+15   3rd Qu.:4.398e+15   3rd Qu.:  1999  
 Max.   :12.000   Max.   :5.395e+15   

In [12]:
data <- na.omit(data)

In [13]:
data <- subset(data, product_mrp!=0)

In [14]:
data$deliverybdays[data$deliverybdays == "\\N"] <- 0
data$deliverycdays[data$deliverycdays == "\\N"] <- 0

#### Outliers

We remove the rows with SLA > 17

In [15]:
data <- data[data$sla < 17,]

In [16]:
data <- data[data$product_procurement_sla < 15,]

In [17]:
data <- data[data$units < 4, ]

#### Dividing the Data into Sub Categories 

In [18]:
table(data$product_analytic_sub_category) 


  AmplifierReceiver      AudioAccessory      AudioMP3Player              Camera 
               4035               11589              112230               98908 
    CameraAccessory       CameraStorage                Game GameMembershipCards 
             234420               17782              108530                  97 
    GamingAccessory       GamingConsole           HomeAudio         HomeTheatre 
             200441               28714              124120                4245 
            Speaker        TVVideoSmall 
             535778              141944 

In [19]:
gaming_accessory <- data[data$product_analytic_sub_category == "GamingAccessory",]
write.csv(gaming_accessory, "gaming_accesory", row.names=F)

In [20]:
camera_accessory <- data[data$product_analytic_sub_category == "CameraAccessory",]
write.csv(camera_accessory, "CameraAccessory", row.names=F)

In [21]:
home_audio <- data[data$product_analytic_sub_category == "HomeAudio",]
write.csv(home_audio, "home_audio", row.names=F)